##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to the Keras Tuner

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [2]:
import io
import re
import string
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization


Install and import the Keras Tuner.

In [3]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 7.5 MB/s 


In [4]:
import keras_tuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Load the data.

In [5]:
url = 'https://github.com/eyalmazuz/AttackAttributionDataset/archive/refs/heads/master.zip' 

dataset = tf.keras.utils.get_file('master.zip', url,extract=True, cache_dir='.',cache_subdir='')

1196032/Unknown - 0s 0us/step

In [6]:
# Normalize pixel values between 0 and 1
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 5
seed = 42

train_ds = tf.keras.utils.text_dataset_from_directory(
    'AttackAttributionDataset-master',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = train_ds.class_names
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

remaining_ds = tf.keras.utils.text_dataset_from_directory(
    'AttackAttributionDataset-master',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = remaining_ds.take(int(len(remaining_ds)*0.5))

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = remaining_ds.skip(len(val_ds))

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 238 files belonging to 12 classes.
Using 191 files for training.
Found 238 files belonging to 12 classes.
Using 47 files for validation.


# Text preprocessing
Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a TextVectorization layer with the desired parameters to vectorize movie reviews. 

In [7]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 100000
sequence_length = 1000

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined [HyperModel](https://keras.io/api/keras_tuner/hypermodels/) classes - [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/) and [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [8]:
min_embedding_dim=16
max_embedding_dim=1024


def model_builder(hp):
  model = keras.Sequential()
  model.add(vectorize_layer)
  hp_units_embed = hp.Int('units_emb_1', min_value=min_embedding_dim, max_value=max_embedding_dim, step=32)
  model.add(Embedding(vocab_size,output_dim= hp_units_embed, name="embedding"))
  model.add(GlobalAveragePooling1D())
 
  # # Tune the number of layers. Best to do it manually for now
  # for i in range(hp.Int("num_layers", 1, 3)):
  #     model.add(
  #         layers.Dense(
  #             # Tune number of units separately.
  #             units=hp.Int(f"units_{i}", min_value=382, max_value=512, step=32),
  #             activation=hp.Choice("activation", ["relu", "tanh"]),
  #         )
  #     )

  # Tune the activation functions in Dense Layers
  hp_activation_func = hp.Choice("activation", ["relu", "tanh"])
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units_1 = hp.Int('units_dense_1', min_value=200, max_value=512, step=10)
  model.add(keras.layers.Dense(units=hp_units_1, activation= hp_activation_func))
  model.add(tf.keras.layers.Dropout(0.25)) 
  hp_units_2 = hp.Int('units_dense_2', min_value=58, max_value=384, step=10)
  model.add(Dense(units=hp_units_2, activation=hp_activation_func))
  model.add(tf.keras.layers.Dropout(0.25)) 
  hp_units_3 = hp.Int('units_dense_3', min_value=32, max_value=192, step=10)
  model.add(Dense(units=hp_units_3, activation=hp_activation_func))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(Dense(16, activation=hp_activation_func))
  model.add(Dense(len(class_names)))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 5e-3, 1e-3, 5e-4, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
stop_early_val_accuracy = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [12]:
tuner.search(train_ds, epochs=50, validation_data=val_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is {best_hps.get('units_dense_1')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.""")

Trial 42 Complete [00h 00m 39s]
val_accuracy: 0.6399999856948853

Best val_accuracy So Far: 0.6800000071525574
Total elapsed time: 00h 12m 13s
INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is 280 and the optimal learning rate for the optimizer is 0.001.


## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [21]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
39/39 [==============================] - 1s 19ms/step - loss: 2.4215 - accuracy: 0.2199 - val_loss: 2.5891 - val_accuracy: 0.1200
Epoch 2/50
39/39 [==============================] - 1s 15ms/step - loss: 2.2062 - accuracy: 0.2670 - val_loss: 2.3553 - val_accuracy: 0.1600
Epoch 3/50
39/39 [==============================] - 1s 15ms/step - loss: 1.7294 - accuracy: 0.4084 - val_loss: 2.8066 - val_accuracy: 0.0800
Epoch 4/50
39/39 [==============================] - 1s 15ms/step - loss: 1.4677 - accuracy: 0.5497 - val_loss: 2.0480 - val_accuracy: 0.2800
Epoch 5/50
39/39 [==============================] - 1s 15ms/step - loss: 1.1869 - accuracy: 0.6283 - val_loss: 2.3244 - val_accuracy: 0.2400
Epoch 6/50
39/39 [==============================] - 1s 15ms/step - loss: 0.9972 - accuracy: 0.7382 - val_loss: 2.3190 - val_accuracy: 0.3600
Epoch 7/50
39/39 [==============================] - 1s 15ms/step - loss: 0.8501 - accuracy: 0.7853 - val_loss: 1.9071 - val_accuracy: 0.4000
Epoch 8/50
39

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [24]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_ds, epochs=best_epoch, validation_data=val_ds)

ResourceExhaustedError: ignored

To finish this tutorial, evaluate the hypermodel on the test data.

In [25]:
eval_result = hypermodel.evaluate(test_ds, label_test)
print("[test loss, test accuracy]:", eval_result)

NameError: ignored

The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.

## Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

Also check out the [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) in TensorBoard to interactively tune your model hyperparameters.